# Making vector field plottings in rt2 look nice 

In [1]:
import sys
import numpy as np 
import plotly.graph_objects as go

sys.path.append('..')

In [ ]:
import plotly.io as pio
import plotly.graph_objects as go

fig = go.Figure(data=go.Bar(y=[1, 3, 2]))

try:
    img = pio.to_image(fig, format="png")
    with open("manual_output.png", "wb") as f:
        f.write(img)
    print("Image saved!")
except Exception as e:
    print("Something went wrong:", e)



In [2]:
# Loading my modules
from physics_informed_vector_fields.current_carrying_wire import field_around_current_carrying_wire
from physics_informed_vector_fields.current_carrying_coil import field_around_current_carrying_coil
from physics_informed_vector_fields.magnetic_dipole import field_around_magnetic_diplole

In [3]:
def plotter(inputs, title='Plot of Vector Fields', rescale = False, perspective=[1.3, 1.3, 1.3], save_as = None):
    import numpy as np
    import plotly.graph_objects as go

    # Unpack and stride
    x, y, z, u, v, w = inputs


    # Compute magnitude
    mags = np.sqrt(u**2 + v**2 + w**2)

    # Avoid division by zero
    mags = np.where(mags == 0, 1e-8, mags)
    if rescale == True:
        # Choose your nonlinear scaling
        # E.g. take square root to reduce contrast
        scale_factor = np.sqrt(mags) / mags  # Now all vectors get a shorter length
        # Or use np.log1p(mags) / mags for log-scaling

        # Rescale vectors
        u = u * scale_factor
        v = v * scale_factor
        w = w * scale_factor

    fig = go.Figure(data=go.Cone(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        u=u.flatten(),
        v=v.flatten(),
        w=w.flatten(),
        # colorscale='Plasma',
        cmin=0,
        sizemode="scaled",
        sizeref=2,  
        anchor="tail",
        colorbar=dict(
            x=0.62,         # default is ~1.0, move left to get closer to the plot
            len=0.75,       # shrink the vertical size a bit if needed
            thickness=15,   # optional: make it slimmer
        ),
    ))

    fig.update_layout(
        title=dict(text=title,
                   x=0.5,
                   y=0.75,
                   xanchor='center',
                   font=dict(size=20)
                ),
        scene=dict(
            xaxis=dict(title='X', showbackground=False),
            yaxis=dict(title='Y', showbackground=False),
            zaxis=dict(title='Z', showbackground=False)
        ),
        scene_camera=dict(eye=dict(x=perspective[0], y=perspective[1], z=perspective[2]))
    )
    if save_as:
        fig.write_image(save_as)
    fig.show()


In [4]:
outputs_wire = field_around_current_carrying_wire(40000000, resolution=10)
plotter(outputs_wire, 
        title = 'B Fields Around Current Carrying Wire', 
        rescale = False, 
        perspective = [1.3, 1.3, 1.3])

In [5]:
outputs_coil = field_around_current_carrying_coil(85000000)
plotter(outputs_coil, 
        title = 'B Fields Around \nCurrent Carrying Coil', 
        rescale = False, 
        perspective = [1.5, 1.5, 0.2])

In [9]:
outputs_dipole = field_around_magnetic_diplole(650000, masking=.0)
plotter(outputs_dipole, 
        title = 'B Fields Around Magnetic Dipole', 
        rescale = False, 
        perspective = [1.5, 0, 0.2])